# Data analysis script

## Input
a single file received from running `grim dump` on the payload

## Output
- FILL THIS IN

In [83]:
import struct
import base64
from collections import namedtuple
from typing import Tuple, List
import pandas as pd
import os


## Data opening and parsing
- open a file to bytes
- bytes to lists of data

In [84]:
SEPARATOR = "********\n"
filename = 'data/lall3.cap'
ouptut_dir = 'out2'

In [85]:
def file_to_bytes(filename: str) -> Tuple[bytes, bytes, bytes, bytes, bytes]:
    with open(filename, 'r') as f:
        parts = f.read().split(SEPARATOR)
        if len(parts) != 5:
            print("wrong number of parts. did you remove all extra lines on top and bottom")
        byte_parts = [base64.b64decode(part) for part in parts]
        return tuple(byte_parts)

In [86]:
SlowData = namedtuple('SlowData', ['timestamp', 'humidity', 'temperature', 'grim_voltage', 'grim_current', 'load_cell_voltage', 'load_cell_current', 'bat_voltage', 'bat_current'])
FastData = namedtuple('FastData', ['timestamp', 'accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz', 'pressure'])
ADCData = namedtuple('ADCData', ['timestamp', 'reading'])
# PreIMUData is FastData
PreALTData = namedtuple('PreALTData', ['timestamp', 'pressure', 'temperature'])


In [87]:
slow_units = SlowData('ms', '% humidity', 'degrees C', 'mV', 'mA', 'mV', 'mA', 'mV', 'mA')
fast_units = FastData('ms', 'm/s^2', 'm/s^2', 'm/s^2', 'rad/s',  'rad/s',  'rad/s', 'kPa')
adc_units = ADCData('ms', 'LSB')
pre_altitude_units = PreALTData('ms', 'kPa', 'degrees C')

In [88]:
slow_fmt='IffHHHHHH' # timestamp, humid, temp, (voltage, current) * 3
fast_fmt = 'Ifffffff' # timestamp accxyz, gyro xyz, press
adc_fmt = 'Iiiiiiiiiii' # timestamp + 10 int32s
imu_boost_detect_fmt = fast_fmt
alt_boost_detect_fmt = 'Iff' # timestamp, press, temp

In [89]:
slow_bs, fast_bs, adc_bs, pre_imu_bs, pre_alt_bs = file_to_bytes(filename)

In [90]:
def interpolate_adc(entries: List[List]) -> List[ADCData]:
    l = []
    for i, entry in enumerate(entries[:-1]):
        start_time = entries[i][0]
        period = entries[i+1][0] - entries[i][0]
        per = period / 10.0
        for j, sample in enumerate(entry[1:]): 
            l.append(ADCData(start_time + j * per, sample))
    return l

In [91]:
slow_lists = list(struct.iter_unpack(slow_fmt, slow_bs))
fast_lists = list(struct.iter_unpack(fast_fmt, fast_bs))
adc_lists = list(struct.iter_unpack(adc_fmt, adc_bs))
pre_imu_lists = list(struct.iter_unpack(imu_boost_detect_fmt, pre_imu_bs))
pre_alt_lists = list(struct.iter_unpack(alt_boost_detect_fmt, pre_alt_bs))


In [92]:
def unit_slow_data(l: List) -> SlowData:
    dr = SlowData(*l)
    du = SlowData(dr.timestamp, dr.humidity, dr.temperature, 1.25 * dr.grim_voltage, 1.25 * dr.grim_voltage, 1.25 * dr.load_cell_voltage, 1.25 * dr.load_cell_current, 1.25 * dr.bat_voltage, 1.25 * dr.bat_current)
    return du

In [93]:
slow_data = [unit_slow_data(l) for l in slow_lists]
fast_data = [FastData(*l) for l in fast_lists]
adc_data = interpolate_adc(adc_lists)

# filter out timestamp = 0 (unwritten) entries
# order by timestamp bc the circular buffer may not begin with the earliest entry
pre_imu_data = sorted([FastData(*l) for l in pre_imu_lists if l[0] != 0], key = lambda d : d.timestamp)
pre_alt_data = sorted([PreALTData(*l) for l in pre_alt_lists if l[0] != 0], key = lambda d : d.timestamp)

In [94]:
pre_alt_elapsed = pre_alt_data[-1].timestamp - pre_alt_data[0].timestamp
pre_imu_elapsed = pre_imu_data[-1].timestamp - pre_imu_data[0].timestamp

In [96]:
print(f"IMU Buffer Time: {pre_imu_elapsed} ms")
print(f"ALT Buffer Time: {pre_alt_elapsed} ms")

IMU Buffer Time: 290 ms
ALT Buffer Time: 4990 ms


In [98]:
percent_imu = 250 / pre_imu_elapsed
print(f"Suggest {percent_imu} of current IMU Buffer size")

Suggest 0.8620689655172413


In [ ]:
# calculate average period of each reading thread
print(f"{len(slow_data)} slow entries. {400 / len(slow_data)} second period")
print(f"{len(fast_data)} fast entries. {1000 * 400 / len(fast_data)} ms period")
print(f"{len(adc_data)} adc entries. {1000 * 400 / len(adc_data)} ms period")


20 slow entries. 20.0 second period
5432 fast entries. 73.63770250368188 ms period
87640 adc entries. 4.564125969876769 ms period


In [ ]:
slow_df = pd.DataFrame(slow_data)
fast_df = pd.DataFrame(fast_data)
adc_df = pd.DataFrame(adc_data)

pre_imu_df = pd.DataFrame(pre_imu_data)
pre_alt_df = pd.DataFrame(pre_alt_data)

In [ ]:
def add_units_to_df(df: pd.DataFrame, units):
    new_names = {}
    for unit, (series_name, _) in zip(units, df.items()):
        new_names[series_name] = f"{series_name} ({unit})"
    df.rename(columns=new_names, inplace=True)

In [ ]:
# set T=0 to start of boost accel buffer
start = pre_imu_df['timestamp'][0]

slow_df['timestamp'] = slow_df['timestamp'] - start
fast_df['timestamp'] = fast_df['timestamp'] - start
adc_df['timestamp'] = adc_df['timestamp'] - start

pre_imu_df['timestamp'] = pre_imu_df['timestamp'] - start
pre_alt_df['timestamp'] = pre_alt_df['timestamp'] - start

In [ ]:
add_units_to_df(slow_df, slow_units)
add_units_to_df(fast_df, fast_units)
add_units_to_df(adc_df, adc_units)
add_units_to_df(pre_alt_df, pre_altitude_units)
add_units_to_df(pre_imu_df, fast_units)

,timestamp (ms),accx (m/s^2),accy (m/s^2),accz (m/s^2),gyrox (rad/s),gyroy (rad/s),gyroz (rad/s),pressure (kPa)
0,292,36.959000,56.799999,-41.988998,3.203,0.274,-1.202,97.366989
1,295,40.126999,56.723999,-37.585999,2.360,1.038,0.474,97.366989
2,299,36.466000,55.646999,-34.834000,1.426,2.951,2.129,97.366753
3,302,29.014999,52.167999,-34.785999,0.724,4.902,3.878,97.366753
4,305,19.223000,45.549000,-23.909000,0.471,6.361,5.187,97.366753
...,...,...,...,...,...,...,...,...
5427,20273,2.254000,-1.741000,9.934000,0.031,-0.053,-0.028,97.366776
5428,20277,2.297000,-1.761000,10.021000,0.030,-0.056,-0.020,97.366776
5429,20280,2.263000,-1.722000,10.001000,0.026,-0.051,-0.037,97.366776
5430,20283,2.239000,-1.780000,10.001000,0.024,-0.052,-0.035,97.366776


In [ ]:
if not os.path.exists(ouptut_dir):
    os.makedirs(ouptut_dir)
slow_df.to_csv(ouptut_dir+'/slow.csv', index=False)
fast_df.to_csv(ouptut_dir+'/fast.csv', index=False)
adc_df.to_csv(ouptut_dir+'/adc.csv', index=False)

pre_imu_df.to_csv(ouptut_dir+'/pre_imu.csv', index=False)
pre_alt_df.to_csv(ouptut_dir+'/pre_alt.csv', index=False)